# Projection of simple Term Death Policies

In [ ]:
import math
import pandas

def month_diff(a, b):
    return 12 * (a.year - b.year) + (a.month - b.month)

def elapsed_months(record):
    return month_diff(valdate, record.StartDate)

def start_age_months(record):
    return month_diff(record.StartDate, record.BirthDate)


In [ ]:
valdate = pandas.Timestamp('2020-10-01')

### Preparing the input Data
First the input is read and the contract number is chosen as the index

In [ ]:
contracts = pandas.read_excel('SampleInput.xlsx')
contracts = contracts.set_index('ContractNo')

Two columns are added to the input file:
  * ElapsedMonths: Number of months between StartDate and valdate
  * StartAgeMonths: Age of the insured person at StartDate in months

In [ ]:
contracts['ElapsedMonths'] = contracts.apply(elapsed_months, 'columns')
contracts['StartAgeMonths'] = contracts.apply(start_age_months, 'columns')
contracts

### Defining the projection
The function `project` creates a new dataframe for each record it is given.

In [ ]:
def project(record):
    global grec
    grec = record
    alpha = {'M': 1.7e-5, 'F': 1.5e-5}[record.Sex]
    beta = 0.0081
    survivors = 1
    accum_deaths = 0

    Time = [0]
    AgeMonths = [record.StartAgeMonths]
    Survivors = [survivors]
    AccumDeaths = [accum_deaths]
    Deaths = [None]

    for t in range(1, record.TermMonths - record.ElapsedMonths + 2):
        age_months = record.StartAgeMonths + t
        # use mortality from previous month
        mortality = min(1.0, alpha * math.exp((age_months-1) * beta))
        deaths = survivors * mortality
        survivors -= deaths
        accum_deaths += deaths

        Time.append(t)
        AgeMonths.append(age_months)
        Survivors.append(survivors)
        AccumDeaths.append(accum_deaths)
        Deaths.append(deaths)

    return pandas.DataFrame({
        'ContractNo': [record.name]*len(Time),
        'Time': Time,
        'Survivors': Survivors,
        'AccumDeaths': AccumDeaths,
        'Deaths': Deaths,
        }).set_index(['ContractNo', 'Time'])

To see the projection in action, lets see the result of projecting the first contract:

In [ ]:
project(contracts.iloc[0])

Usually we will project all contracts at once:

In [ ]:
projection = pandas.concat(contracts.apply(project, 'columns').tolist())

Then lets get the SumInsured from the original table and calculate a cashflow DeathClaim

In [ ]:
projection = projection.merge(contracts['SumInsured'], how='left', left_index=True, right_index=True)
projection['DeathClaim'] = projection['SumInsured'] * projection['Deaths']
projection

### Discount Rate
For simplicity a flat rate

In [ ]:
time = list(range(1000))
df = [1.02 ** (-t/12) for t in time]
discount = pandas.DataFrame({'Time': time, 'DiscountFactor': df})
discount = discount.set_index('Time')
discount

In [ ]:
projection = projection.merge(discount, how='left', left_index=True, right_index=True)
projection

### Aggregating projected Cashflows
The cashflows are aggregated by contract and then added as a new column in the contracts table

In [ ]:
contracts['TotalDeathClaim'] = projection.groupby(['ContractNo'])[['DeathClaim']].sum()
contracts['DiscountedDeathClaim'] = projection.assign(col=projection['DiscountFactor']*projection['DeathClaim']).groupby(['ContractNo']).col.sum()
contracts